In [1]:
from oct_onh.opticdisc import OpticDiscOCTEnfaceGroup, OpticDiscCircularBScan, OpticDiscRadialBScan, OpticDiscBscan
from oct_onh.octbscan import OCTBScan
import numpy as np

## Description of how to create the OCT objects

In [3]:
## Load the b-scan from file
## E.g. image = np.array(Image.open('path/to/image.png'))
## Now using random noise for demonstration
image = np.random.rand(512, 512)
## Load the enface image (e.g. IR localizer) from file
## E.g. enface_image = np.array(Image.open('path/to/enface_image.png'))
enface_image = np.random.rand(512, 512)

## Provide metadata: resolution in mm and laterality as 'L' or 'R'
bscan = OCTBScan(
    imagedata=image,
    res_width_mm = 0.01, ## along the x-axis of the b-scan
    res_height_mm = 0.0038, ## along the y-axis of the b-scan
    rows_y=image.shape[0],
    columns_x=image.shape[1],
    laterality = "R"
    )

## If the b-scan is a circular scan, provide the center and radius
circular_bscan = OpticDiscCircularBScan(
    bscan=bscan,
    mask=np.random.rand(512,512), center=(200,200),
    radius= 100, start_angle=np.pi, real_diameter=1.0
)

## If the b-scan is a radial scan, provide the start and end point in enface coordinates and the resolution of the enface image
## This is used to calculate the angle of the scan and location of BMO endpoints
radial_scan = OpticDiscRadialBScan(mask=np.random.rand(512,512), bscan=bscan, startpoint=(1,1), endpoint=(10,10),
                                        res_y_enface=0.001,
                                        res_x_enface=0.001)

## Combine any number of b-scans into a group to extract combined features
grouped = OpticDiscOCTEnfaceGroup(enface_image=enface_image, bscans=[radial_scan, circular_bscan], 
                                       laterality='L', enface_resolution=0.001)

## Loading the example from file

In [2]:
import json
with open('../resources/heidelberg_example/locations.json', 'r') as f:
    locations = json.load(f)

In [3]:
from PIL import Image

## Using the example data
bscans = []
for i, locs in enumerate(locations):
    path = f'../resources/heidelberg_example/OCT-0_{i}.png'
    image = np.array(Image.open(path))
    bscan = OCTBScan(
        imagedata=image,
        res_width_mm = 0.0057, ## along the x-axis of the b-scan
        res_height_mm = 0.0038, ## along the y-axis of the b-scan
        rows_y=image.shape[0],
        columns_x=image.shape[1],
        laterality = "R"
    )
    if 'start' in locs:
        optic_disc_scan = OpticDiscRadialBScan(bscan=bscan, 
                                            startpoint=(locs['start']['x'], locs['start']['y']), 
                                            endpoint=(locs['end']['x'], locs['end']['y']),
                                            res_y_enface=0.0057, res_x_enface=0.0057)
    else:
        optic_disc_scan = OpticDiscCircularBScan(bscan=bscan,
                                            center=(locs['centre']['x'], locs['centre']['y']),
                                            radius=locs['radius'], start_angle=locs['start_angle'],
                                            real_diameter=0.0057*locs['radius']*2)
    bscans.append(optic_disc_scan)

enface_image = np.array(Image.open('../resources/heidelberg_example/SLO_IR-0_0.png'))
grouped = OpticDiscOCTEnfaceGroup(enface_image=enface_image, bscans=bscans, laterality='R', enface_resolution=0.0057)
    

## Load the masks, or...

In [7]:
from oct_onh.loading_utils import load_gzip_binary

segmentation = load_gzip_binary('../resources/heidelberg_example/segmentation.binary.gz', image.shape[0], image.shape[1])

## ... apply the model

In [ ]:
from oct_onh import model
segmodel = model.ONHSegmentationModel(path = '/path/to/model/files/nnUNetTrainer__nnUNetPlans__2d/')

bscan_volume = np.array([b.image for b in grouped.bscans])
segmentation = segmodel.predict(bscan_volume)[0]


/home/kvangarderen/rtnls-env/lib/python3.11/site-packages/nnunetv2/utilities/plans_handling/plans_handler.py:37: UserWarning: Detected old nnU-Net plans format. Attempting to reconstruct network architecture parameters. If this fails, rerun nnUNetv2_plan_experiment for your dataset. If you use a custom architecture, please downgrade nnU-Net to the version you implemented this or update your implementation + plans.
  warnings.warn("Detected old nnU-Net plans format. Attempting to reconstruct network architecture "


nnUNet_raw is not defined and nnU-Net can only be used on data for which preprocessed files are already present on your system. nnU-Net cannot be used for experiment planning and preprocessing like this. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up properly.
nnUNet_preprocessed is not defined and nnU-Net can not be used for preprocessing or training. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up.

Predicting image of shape torch.Size([1, 2, 512, 512]):
perform_everything_on_device: True


100%|██████████| 4/4 [00:00<00:00, 45.37it/s]


sending off prediction to background worker for resampling

Done with image of shape torch.Size([1, 2, 512, 512]):


## Add the masks to the bscan objects

In [ ]:
for mask, bscan in zip(segmentation, grouped.bscans):
    bscan.mask = mask

## Extract features

In [6]:
import matplotlib.pyplot as plt

grouped.get_all_features()


{'RNFL thickness d=3.4 mm segment T (mm) mean': 0.07643316062176166,
 'RNFL thickness d=3.4 mm segment T (mm) n points': 193,
 'RNFL thickness d=3.4 mm segment TS (mm) mean': 0.11377647058823527,
 'RNFL thickness d=3.4 mm segment TS (mm) n points': 85,
 'RNFL thickness d=3.4 mm segment NS (mm) mean': 0.11672705882352946,
 'RNFL thickness d=3.4 mm segment NS (mm) n points': 85,
 'RNFL thickness d=3.4 mm segment N (mm) mean': 0.08982553191489362,
 'RNFL thickness d=3.4 mm segment N (mm) n points': 235,
 'RNFL thickness d=3.4 mm segment NI (mm) mean': 0.12611529411764702,
 'RNFL thickness d=3.4 mm segment NI (mm) n points': 85,
 'RNFL thickness d=3.4 mm segment TI (mm) mean': 0.1533411764705883,
 'RNFL thickness d=3.4 mm segment TI (mm) n points': 85,
 'RNFL thickness d=3.4 mm hour 0 (mm) mean': 0.1246875,
 'RNFL thickness d=3.4 mm hour 0 (mm) n points': 64,
 'RNFL thickness d=3.4 mm hour 1 (mm) mean': 0.106340625,
 'RNFL thickness d=3.4 mm hour 1 (mm) n points': 64,
 'RNFL thickness d=3.